In [71]:
import os
import streamlit as st
import pickle
import time
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms import HuggingFaceHub

In [5]:
# Initialise Hugging Face transformer pipeline with a model from Hugging Face Hub
hf_llm = pipeline("text-generation", model="gpt2") 

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [72]:
# Initialize Hugging Face model through LangChain's HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "api-key"  
# Initialize the FLAN-T5 model via HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large", 
    model_kwargs={"temperature": 0.7, "max_new_tokens": 200}
)

In [73]:
# Loaders for the source data
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()
print("Loaded data length:", len(data))

Loaded data length: 2


In [74]:
# Split the text data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [75]:
# Create embeddings for each chunk using Hugging Face's embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [76]:
# Create a FAISS vector index from the document chunks and embeddings
vectorindex = FAISS.from_documents(docs, embeddings)

In [77]:
# Save the vector index locally
file_path = "vector_index_hg.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex, f)

In [78]:
#Load the vector index if it exists
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [79]:
# Instantiate the chain using HuggingFaceHub and the vector retriever
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())

In [80]:
query = "What is the price of Tiago iCNG?"
result = chain({"question": query}, return_only_outputs=True)
print("Answer:", result)

Answer: {'answer': '', 'sources': 'The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.'}
